In [1]:
import os
from dotenv import load_dotenv
import logging
from bson import json_util

from pymongo import MongoClient
from bson import ObjectId

load_dotenv()
logging.basicConfig(level=logging.INFO)

In [2]:
class MongoConnection:
    def __init__(self, collection_name: str, timeout_ms: int = 5000):
        self.uri = os.getenv("MONGO_URI")
        self.timeout_ms = timeout_ms
        self.db_name = os.getenv("DATABASE_NAME")
        self.collections = collection_name
        
    def connect(self) -> MongoClient:
        if not self.uri:
            raise ValueError("MONGO_URI is not set")
        self.client = MongoClient(self.uri, serverSelectionTimeoutMS=self.timeout_ms)
        self.client.admin.command("ping")
        logging.info("Mongo connected")
        return self.client
    
    def get_db(self):
        if not self.db_name or not self.collections:
            raise ValueError("DATABASE_NAME or COLLECTIONS is not set")
        self.db = self.client[self.db_name]
        self.collection = self.db[self.collections]
        return self.db, self.collection

In [ ]:
mongo_sensor = MongoConnection(collection_name="SENSOR_COLLECTION")
client = mongo_sensor.connect()
db, collection = mongo_sensor.get_db()

INFO:root:Mongo connected


In [4]:
try:
    sensor_oid = ObjectId("665feb0321d9dc54daa4e439")
    query = {"_id": sensor_oid}
except Exception as e:
    logging.error(f"Error : {e}")

In [5]:
result = collection.find_one(query)
if result:
    print("Sensor document found:")
    print(json_util.dumps(result, indent=4))
else:
    print(f"No sensor found with ID {sensor_oid}")

No sensor found with ID 665feb0321d9dc54daa4e439
